# qPCR Data Analysis Notebook


- This notebook extracts data from Applied Biosystems qPCR results spreadsheets and proceeds to calculate 2^-dCT and 2^-ddCT.
- User input of folder path, target genes, housekeeping gene, and control sample (for 2^-ddCT).
- The script filters out outliers that are >1 away from the technical replicate median.
- Notebook should run in virtual environment with the packages and package versions stated in the .yaml file.

## 1. Import libraries and modules

In [5]:
# check what libraries are installed in conda environment
!conda list

# packages in environment at /opt/anaconda3/envs/data_analysis:
#
# Name                    Version                   Build  Channel
absl-py                   2.1.0                    pypi_0    pypi
appnope                   0.1.4              pyhd8ed1ab_0    conda-forge
asttokens                 2.4.1              pyhd8ed1ab_0    conda-forge
astunparse                1.6.3                    pypi_0    pypi
bzip2                     1.0.8                h6c40b1e_6  
ca-certificates           2024.7.4             h8857fd0_0    conda-forge
certifi                   2024.7.4                 pypi_0    pypi
charset-normalizer        3.3.2                    pypi_0    pypi
comm                      0.2.2              pyhd8ed1ab_0    conda-forge
contourpy                 1.2.1                    pypi_0    pypi
cycler                    0.12.1                   pypi_0    pypi
debugpy                   1.8.3           py312h28f332c_0    conda-forge
decorator                 5.1.1              p

In [1]:
from matplotlib import pyplot as plt 
import pandas as pd
import numpy as np
import scipy as sp 
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

Matplotlib is building the font cache; this may take a moment.
Fontconfig warning: ignoring UTF-8: not a valid region tag
